In [1]:
%load_ext watermark
%watermark -v -n -m -p numpy,sklearn,pandas,neptune

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.4.0

numpy  : 1.23.2
sklearn: 1.1.2
pandas : 1.4.3
neptune: 0.16.5

Compiler    : GCC 10.4.0
OS          : Linux
Release     : 5.15.0-46-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from datetime import datetime

# kernels
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel


PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/fedesss/Dropbox/ML_to_ETa/eta_ml


In [9]:
def get_scaled_data():
    df = pd.read_pickle(PROJ_ROOT + '/data/interim/scaled.pickle')
    return df

def get_processed_data():
    df = pd.read_pickle(PROJ_ROOT + '/data/processed/' + 'processed.pickle')
    return df

scaled = get_scaled_data()
df = get_processed_data()

print(len(scaled))
print(len(df))

4352
4285


In [3]:
 def make_kernel(**params):
     # Long term rising trend
     length_scale = params.get('rbf_len')
     long_term_trend_kernel = 50.0**2 * RBF(length_scale=length_scale)
     # Seasonal variation
     length_scale = params.get('sine_len')
     seasonal_kernel = (
         2.0**2
         * RBF(length_scale=100.0)
         * ExpSineSquared(length_scale=length_scale, periodicity=1.0, periodicity_bounds="fixed")
     )
     # Small irregularities
     length_scale = params.get('irreg_len')
     irregularities_kernel = 0.5**2 * RationalQuadratic(length_scale=length_scale, alpha=1.0)
     # Noise
     noise_level = params.get('noise_level')
     noise_bounds = params.get('noise_bounds')
     noise_kernel = 0.1**2 * RBF(length_scale=0.1) + WhiteKernel(
         noise_level=noise_level, noise_level_bounds=noise_bounds
     )
     return long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel


In [4]:
 def kernel_params():
     kernel_params = {
         'rbf_len': 100.0,
         'sine_len': 1.0,
         'irreg_len': 1.0,
         'noise_level': 0.1**2,
         'noise_bounds': (1e-5, 1e5),
     }
     return kernel_params
 
 def model_params(kernel):
     params = {
         'kernel': kernel,
         'alpha': 1e-10,
         'optimizer': 'fmin_l_bfgs_b',
         'n_restarts_optimizer': 1,
         'normalize_y':False,
     }
     return params

kernel_params = kernel_params()
params = model_params(make_kernel(**kernel_params))
params['seed'] = 1998

In [5]:
kernel_params

{'rbf_len': 100.0,
 'sine_len': 1.0,
 'irreg_len': 1.0,
 'noise_level': 0.010000000000000002,
 'noise_bounds': (1e-05, 100000.0)}

In [6]:
params

{'kernel': 50**2 * RBF(length_scale=100) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) + 0.1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=0.01),
 'alpha': 1e-10,
 'optimizer': 'fmin_l_bfgs_b',
 'n_restarts_optimizer': 1,
 'normalize_y': False,
 'seed': 1998}